# BayesDB analysis of fish bouts in relation to paramecia characteristics

## 1. Ingesting the data

### a. Launching BayesDB

In [24]:
%load_ext jupyter_probcomp.magics

The jupyter_probcomp.magics extension is already loaded. To reload it, use:
  %reload_ext jupyter_probcomp.magics


In [25]:
%matplotlib inline
%vizgpm inline

<IPython.core.display.Javascript object>

In [26]:
!rm -f 042318_6/bdb_hunts_inverted.bdb
%bayesdb -j 042318_6/bdb_hunts_inverted.bdb

u'Loaded: 042318_6/bdb_hunts_inverted.bdb'

### b. Loading and printing the data from a `.csv` file

In [27]:
%sql DROP TABLE IF EXISTS bout_table;
%bql CREATE TABLE bout_table FROM '042318_6/huntbouts_inverted.csv' 

""


In [28]:
%sql SELECT * FROM bout_table LIMIT 25;

,Bout Number,Bout Az,Bout Alt,Bout Dist,Bout Delta Pitch,Bout Delta Yaw,Para Az,Para Alt,Para Dist,Para Az Velocity,Para Alt Velocity,Para Dist velocity,Postbout Para Az,Postbout Para Alt,Postbout Para Dist,Strike Or Abort
0,1.0,1.982416,0.299406,55.543836,0.066884,0.494285,0.637470,0.632546,173.949790,0.004821,-0.004269,-1.182665,-0.156973,0.444844,161.943651,1.0
1,1.0,0.659735,0.193954,37.535483,0.035177,0.171169,0.084501,0.328278,243.871697,-0.002494,-0.000956,0.474774,0.125327,0.281939,219.718842,2.0
2,2.0,0.026787,0.116177,42.293048,-0.005488,0.016661,0.123433,0.263269,212.646078,-0.001821,-0.004629,-0.596787,0.107701,0.246955,178.601562,2.0
3,3.0,0.434611,0.058352,58.808604,0.029150,0.176588,0.117435,0.245743,167.145417,0.001698,-0.001300,-2.282867,-0.252801,0.164426,129.049859,2.0
4,4.0,0.426897,-0.049720,64.337897,-0.000734,0.169667,0.302955,0.094228,113.713507,-0.007594,-0.018352,-0.128023,-0.013440,-0.000810,72.736933,2.0
5,1.0,0.273788,0.409306,31.035482,0.042746,0.087416,0.088988,0.426451,206.439055,-0.008272,-0.001612,-0.082692,0.015776,0.383671,184.321178,1.0
6,2.0,0.295023,0.304067,22.986699,0.035820,0.043896,0.021460,0.384433,177.920890,0.001165,0.000561,-1.093454,-0.034537,0.365724,163.025697,1.0
7,3.0,0.124791,0.204198,33.469063,0.039520,0.032196,0.028319,0.370375,155.716037,-0.002594,0.001457,-2.455775,0.077434,0.371098,135.051601,1.0
8,4.0,0.176450,0.335897,56.468441,0.098241,0.021631,0.084984,0.372589,133.390374,0.008228,0.002651,-2.423232,0.170305,0.213914,98.628971,1.0
9,5.0,0.471212,0.051754,62.677898,0.092949,0.265652,0.199475,0.134826,90.869249,0.005501,-0.007214,-0.366207,0.063720,0.135139,57.822733,1.0


In [29]:
%bql .nullify bout_table ''
%bql .nullify bout_table 'nan'

Nullified 0 cells
Nullified 0 cells


## 2. Automatically learning a CrossCat probabilistic model

### a. Defining an analysis population

In [30]:
%bql GUESS SCHEMA FOR bout_table

,column,stattype,num_distinct,reason
0,Bout Number,nominal,8.0,There are fewer than 20 distinct numerical va...
1,Bout Az,numerical,36.0,There are at least 20 unique numerical values...
2,Bout Alt,numerical,36.0,There are at least 20 unique numerical values...
3,Bout Dist,numerical,36.0,There are at least 20 unique numerical values...
4,Bout Delta Pitch,numerical,36.0,There are at least 20 unique numerical values...
5,Bout Delta Yaw,numerical,36.0,There are at least 20 unique numerical values...
6,Para Az,numerical,36.0,There are at least 20 unique numerical values...
7,Para Alt,numerical,36.0,There are at least 20 unique numerical values...
8,Para Dist,numerical,36.0,There are at least 20 unique numerical values...
9,Para Az Velocity,numerical,36.0,There are at least 20 unique numerical values...


In [31]:
%%mml
DROP POPULATION IF EXISTS bout_population;
CREATE POPULATION bout_population FOR bout_table WITH SCHEMA (GUESS STATTYPES OF (*);
                          IGNORE "Bout Number";)

""


### b. Creating and analyzing a probabilistic model (automatically)

In [32]:
%mml CREATE GENERATOR huntbouts_crosscat FOR bout_population;

""


In [33]:
%mml INITIALIZE 100 MODELS IF NOT EXISTS FOR huntbouts_crosscat;

""


In [34]:
%mml ANALYZE huntbouts_crosscat FOR 100 ITERATIONS (OPTIMIZED);
# note you can use the (OPTIMIZED) flag here but that gave you weird results last time. 

Completed: 100 iterations in 3.586695 seconds.
Completed: 100 iterations in 3.852431 seconds.
Completed: 100 iterations in 3.548502 seconds.
Completed: 100 iterations in 4.462598 seconds.
Completed: 100 iterations in 3.718305 seconds.
Completed: 100 iterations in 4.607347 seconds.
Completed: 100 iterations in 2.541449 seconds.
Completed: 100 iterations in 5.161964 seconds.
Completed: 100 iterations in 2.758186 seconds.
Completed: 100 iterations in 3.306801 seconds.
Completed: 100 iterations in 3.637106 seconds.
Completed: 100 iterations in 2.541877 seconds.
Completed: 100 iterations in 2.317847 seconds.
Completed: 100 iterations in 4.078185 seconds.
Completed: 100 iterations in 2.359556 seconds.
Completed: 100 iterations in 4.272125 seconds.
Completed: 100 iterations in 2.166239 seconds.
Completed: 100 iterations in 2.422345 seconds.
Completed: 100 iterations in 5.420803 seconds.
Completed: 100 iterations in 3.601082 seconds.
Completed: 100 iterations in 4.114598 seconds.
Completed: 10

""
